# Предсказание согласия на заявку при условии звонка
#### Задача https://jira.tcsbank.ru/browse/DMP-3713

Исходные данные:
   * Выборка звонков лежит в big_data_science.aza_to_liru_170627 . Период с конца 2015 года по середину 2017.

In [15]:
##### Config

import datetime
import pandas as pd
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import hashlib
from collections import Counter

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''
try:
    sc.stop()
except NameError:
    pass

conf = (SparkConf()
        .set("spark.executor.instances", 4)
        .set("spark.driver.maxResultSize", "8g")
        .set('spark.driver.memory','8g')
        .set("spark.executor.memory", '4g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
        .set("spark.akka.frameSize", '1024')        
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

### Привязываем лиру куки и создаем таблицу с визитами

In [1]:
queries = '''

create table user_kposminin.full_app_from_short_id as 
select
    concat('+7', substr(a.phone_mobile,2,20)) as phone_num,
    m.id,
    cast(a.create_dt as date) as call_ymd, 
    a.wo_task_reason_desc as reason_desc,
    a.approve
from
    big_data_science.aza_to_liru_170627 a
    inner join
    (select 
       uid_str as id,
       property_value as phone_num,
       load_src
     from
       prod_dds.md_uid_property 
     where
       property_cd = 'PHONE'
    ) m on concat('+7', substr(a.phone_mobile,2,20)) = m.phone_num
;

create table user_kposminin.full_app_from_short_visits (
       phone_num string,
       call_ymd string,
       url_fragemtn string,
       duration_sec int,
       visit_count int,
       average_visit_hour int,
       load_src string,
       visit_ymd string
)
stored as RCFile
;
'''

In [12]:
visits_query_parttern = '''
insert into user_kposminin.full_app_from_short_visits 
select
       a.phone_num,
       a.call_ymd,
       v.url_fragment,
       v.duration_sec, 
       v.visit_count, 
       v.average_visit_hour, 
       v.load_src, 
       v.ymd as visit_ymd
  from user_kposminin.full_app_from_short_id a
 inner join prod_odd.visit_feature v on v.id = a.id
 where v.ymd = \\'#ymd0\\'
   and a.call_ymd between date_add(\\'#ymd0\\',1) and date_add(\\'#ymd0\\', 180)
      
'''
open('./scripts/full_app_from_short_visits_query_parttern.sql','w').write(visits_query_parttern.replace('\n',' ') + '\n')

In [6]:
! seq 800 | xargs -I {} date -d '20170531 -{} days' +%Y-%m-%d | xargs -I {} sed -e 's/#ymd0/{}/g' ./scripts/full_app_from_short_visits_query_parttern.sql | xargs -P 16 -I '{}' sh -c 'hive -e "{}" &>/dev/null'

### Как-то плохо работает. Непараллелится и часть дней не посчитал

In [ ]:
import datetime, os
ymd_loaded = [d[6:]+'-'+d[3:5] + '-' + d[:2] for d in ymd_loaded_str.split('\n')[1:-1]]
c=0
for d in [(datetime.date(2017,5,31) + datetime.timedelta(days = -i)).strftime('%Y-%m-%d') for i in range(800)]:
    if not d in ymd_loaded:
        c+=1
        query = visits_query_parttern1.replace('#ymd0',d)
        print(query + ';\n')
        #os.popen("hive -e '" + query + "' &>/dev/null").read()